In [1]:
# dependencies
import os
#clear Screen
os.system("cls")

import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, join, outerjoin, MetaData, Table

# create engine to hawaii.sqlite
connect_string = "sqlite:///static/data/climateDB.db"

# reflect the tables
engine = create_engine(connect_string) 

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect =True)

# View all of the classes that automap found
print(Base.classes.keys())

# Save references to each table
Emission = Base.classes.CO2_emission
Temp_change = Base.classes.temp_change
Country_demo = Base.classes.country_demo
#print(Emission)

# Create a session (link) from Python to the sqlite DB
session = Session(bind = engine)






['CO2_emission', 'country_demo', 'temp_change']


In [2]:
#Filter the data for the year >= 1961
results_emission = session.query(Emission).filter(Emission.Year >= 1961)
emission_df = pd.read_sql(results_emission.statement, session.connection())

#print(emission_df.head())
results_temp = session.query(Temp_change)
temp_df = pd.read_sql(results_temp.statement, session.connection())

selection = ['DecJanFeb', 'MarAprMay', 'JunJulAug', 'SepOctNov']
season_df = temp_df.loc[temp_df["Months"].isin(selection)]



In [3]:
#filter by months
month_df = temp_df.loc[(~temp_df["Months"].isin(selection)) & (temp_df["Months"] != 'Meteorological year') ]

# filter Meteorological year
meteor_df = temp_df.loc[temp_df["Months"] == 'Meteorological year' ]
meteor_df_new = meteor_df.copy()

meteor_df_new.head()

,field1,Area,Months,Element,Unit,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
16,32,Afghanistan,Meteorological year,Temperature change,°C,-0.080,-0.120,0.882,-0.727,-0.200,...,1.564,1.342,0.102,1.237,0.474,1.096,1.473,1.483,1.524,0.893
33,66,Albania,Meteorological year,Temperature change,°C,0.631,0.339,0.084,-0.171,-0.392,...,1.294,1.123,1.603,1.495,1.399,1.725,1.697,1.337,2.232,1.911
50,100,Algeria,Meteorological year,Temperature change,°C,0.186,0.101,0.045,0.212,-0.135,...,2.359,1.492,1.238,1.267,1.774,1.224,1.866,1.630,1.326,1.224
67,168,Andorra,Meteorological year,Temperature change,°C,0.749,0.110,-0.755,0.305,-0.500,...,0.441,1.643,1.233,0.794,1.899,1.619,1.917,1.853,1.826,1.871
84,202,Angola,Meteorological year,Temperature change,°C,0.053,-0.139,-0.176,-0.215,-0.181,...,1.058,0.708,0.401,0.910,0.726,1.260,1.534,0.808,1.264,1.694


In [4]:
# Calculate avg temp per Meteorological year
meteor_df_new['avg_temp']= round(meteor_df_new.mean(axis =1),3)
meteor_df_new.head()

,field1,Area,Months,Element,Unit,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,avg_temp
16,32,Afghanistan,Meteorological year,Temperature change,°C,-0.080,-0.120,0.882,-0.727,-0.200,...,1.342,0.102,1.237,0.474,1.096,1.473,1.483,1.524,0.893,0.958
33,66,Albania,Meteorological year,Temperature change,°C,0.631,0.339,0.084,-0.171,-0.392,...,1.123,1.603,1.495,1.399,1.725,1.697,1.337,2.232,1.911,1.577
50,100,Algeria,Meteorological year,Temperature change,°C,0.186,0.101,0.045,0.212,-0.135,...,1.492,1.238,1.267,1.774,1.224,1.866,1.630,1.326,1.224,2.366
67,168,Andorra,Meteorological year,Temperature change,°C,0.749,0.110,-0.755,0.305,-0.500,...,1.643,1.233,0.794,1.899,1.619,1.917,1.853,1.826,1.871,3.480
84,202,Angola,Meteorological year,Temperature change,°C,0.053,-0.139,-0.176,-0.215,-0.181,...,0.708,0.401,0.910,0.726,1.260,1.534,0.808,1.264,1.694,3.773


In [6]:

meteor_id_df = meteor_df_new.set_index('Area')

results_Demo = session.query(Country_demo)
demo_df = pd.read_sql(results_Demo.statement, session.connection(),index_col='index')

demo_sorted_df = demo_df.sort_values(by=["name"]).reset_index(drop=True)
demo_sorted_df = demo_sorted_df.rename({"name":"Area","population":"Population",
                                        "density":"Density",
                                        "land-size":"Land Size","image_url":"Images",
                                        "latitude":"Lat","longitude":"Lng"},axis='columns')

session.close()

meteor_id_df.head()

,field1,Months,Element,Unit,1961,1962,1963,1964,1965,1966,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,avg_temp
Area,,,,,,,,,,,,,,,,,,,,,
Afghanistan,32,Meteorological year,Temperature change,°C,-0.080,-0.120,0.882,-0.727,-0.200,0.176,...,1.342,0.102,1.237,0.474,1.096,1.473,1.483,1.524,0.893,0.958
Albania,66,Meteorological year,Temperature change,°C,0.631,0.339,0.084,-0.171,-0.392,0.551,...,1.123,1.603,1.495,1.399,1.725,1.697,1.337,2.232,1.911,1.577
Algeria,100,Meteorological year,Temperature change,°C,0.186,0.101,0.045,0.212,-0.135,0.404,...,1.492,1.238,1.267,1.774,1.224,1.866,1.630,1.326,1.224,2.366
Andorra,168,Meteorological year,Temperature change,°C,0.749,0.110,-0.755,0.305,-0.500,0.403,...,1.643,1.233,0.794,1.899,1.619,1.917,1.853,1.826,1.871,3.480
Angola,202,Meteorological year,Temperature change,°C,0.053,-0.139,-0.176,-0.215,-0.181,0.193,...,0.708,0.401,0.910,0.726,1.260,1.534,0.808,1.264,1.694,3.773


In [7]:
meteor_id_df.describe()

,field1,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,avg_temp
count,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.00000,201.000000,201.000000,...,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000
mean,4973.840796,0.187935,-0.020040,-0.025388,-0.105488,-0.240428,0.113025,-0.10992,-0.215512,0.116249,...,0.841075,0.895612,0.935816,1.115294,1.262229,1.461821,1.306279,1.280423,1.412134,83.356164
std,2887.726728,0.415328,0.366187,0.419395,0.303493,0.263760,0.435902,0.34965,0.346749,0.351845,...,0.406638,0.461490,0.307866,0.540317,0.448338,0.461735,0.421351,0.554280,0.426632,48.142899
min,32.000000,-0.688000,-1.364000,-2.245000,-0.871000,-1.058000,-2.357000,-1.01300,-3.252000,-1.740000,...,-0.232000,-0.216000,0.106000,0.180000,-0.095000,0.217000,0.154000,0.332000,0.275000,0.958000
25%,2684.000000,-0.080000,-0.177000,-0.208000,-0.261000,-0.392000,-0.046000,-0.28100,-0.330000,-0.069000,...,0.530000,0.618000,0.740000,0.750000,1.004000,1.221000,1.067000,0.894000,1.175000,45.150000
50%,4860.000000,0.052000,-0.084000,0.013000,-0.110000,-0.210000,0.109000,-0.15600,-0.191000,0.190000,...,0.806000,0.800000,0.936000,0.993000,1.234000,1.423000,1.294000,1.142000,1.413000,81.625000
75%,7716.000000,0.356000,0.118000,0.206000,0.079000,-0.086000,0.305000,0.03200,-0.065000,0.334000,...,1.074000,1.139000,1.098000,1.360000,1.471000,1.697000,1.533000,1.519000,1.627000,128.906000
max,9654.000000,1.906000,1.044000,1.174000,1.121000,0.856000,1.426000,1.13900,0.482000,0.825000,...,2.106000,3.825000,2.272000,3.382000,3.307000,5.413000,3.347000,4.104000,2.903000,161.386000


In [ ]:
##===================================================================##
##Functions
##====================================================================##

## Return launchPage

# def launchPage() :   

    #calculate overall avg_co2 emission per country
    avg_co2 =  emission_df.groupby("Entity").agg({'AnnualCO2emissions':'mean'})
    avg_co2 = round(avg_co2/1000000,3) ## converting GT to Mega ton for the tooltip
    avg_co2.reset_index(inplace=True)
    avg_co2 = avg_co2.rename({'Entity':'Area', 'AnnualCO2emissions':'AnnualCO2emissions'}, axis='columns')

    #Merge Temp_change by meteor year per country to Avg_Co2 Emission df
    merged_co2_country =meteor_id_df.merge(avg_co2, how = 'inner',  on="Area")

    #find null
    merged_co2_country.isna().sum()
    #fill 0
    merged_co2_country = merged_co2_country.fillna(0)
    #find null again
    merged_co2_country.isna().sum()

    #merge population data to Temp and Co2 Emission df
    popu_data = merged_co2_country.merge(demo_sorted_df, how = 'left', on ="Area" )

    popu_data = popu_data[['Area', 'Population','Density', 'Land Size', 'Images', 'Lat','Lng']].fillna(0).set_index("Area",drop=True)
    merged_co2_country.set_index("Area",inplace=True)

    #Get New Countries from the merged DF
    New_Countries = merged_co2_country.index

    #Create a dictionary holding above values
     #meta = [{
    #    'country' : country_name,
    #    'demo_info' : [web scraped data],
    #    'tool_tip' : [{'c_name': country_name,
    #                'avg_temp':avg_temp,
    #                'avg_co2' : new_Avg_c02,
    #                'population':population from web scraping
    #                }]
    #   } ]

    #New Code---------------------------------------------------

    meta = []

    for country in New_Countries:
        temp_co2_obj = {
                        "Country":country,
                        "Avg Temp Change":merged_co2_country.loc[country,"avg_temp"],
                        "Avg Co2 Change":merged_co2_country.loc[country,"AnnualCO2emissions"],
                        "Population":popu_data.loc[country,"Population"],
                        "Density":popu_data.loc[country,"Density"],
                        "Land Size":popu_data.loc[country,"Land Size"],
                        "Images":popu_data.loc[country,"Images"],
                        "Lat":popu_data.loc[country,"Lat"],
                        "Lng":popu_data.loc[country,"Lng"],
                        }
        meta.append(temp_co2_obj)

#     return meta , New_Countries